# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install dependencies

In [ ]:
!pip install -q diffusers transformers accelerate ftfy einops xformers torchmetrics Pillow tqdm peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 32.2 MB/s eta 0:00:00


# Imports

In [ ]:
import torch
import os
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from peft import PeftModel
from pathlib import Path
from PIL import Image
from tqdm import tqdm

# Config

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BASE_MODEL = "runwayml/stable-diffusion-v1-5"

# LoRA paths
LORA_ROOT = "/content/drive/MyDrive/Colab Notebooks/sd_lora"
UNET_LORA_WEIGHTS = f"{LORA_ROOT}/outputs/unet_te/checkpoint-step500/unet"
TEXT_ENCODER_LORA_WEIGHTS = f"{LORA_ROOT}/outputs/unet_te/checkpoint-step500/text_encoder"

# Output paths
OUTPUT_DIR = f"{LORA_ROOT}/testing/comparison_test/unet_te/checkpoint-step500"
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

# Generation settings
PROMPTS = [
    "Sarma, a traditional Serbian dish of cabbage rolls served in a clay pot",
    "Ćevapi served with flatbread and onions"
]
RESOLUTION = 512
NUM_INFERENCE_STEPS = 30
GUIDANCE_SCALE = 7.5

# Load base Stable Diffusion

In [ ]:
pipe_base = StableDiffusionPipeline.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    safety_checker=None
).to(DEVICE)

pipe_base.scheduler = DPMSolverMultistepScheduler.from_config(pipe_base.scheduler.config)
pipe_base.enable_xformers_memory_efficient_attention()
pipe_base.enable_attention_slicing()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


# Load LoRA-enhanced Stable Diffusion

In [ ]:
pipe_lora = StableDiffusionPipeline.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    safety_checker=None
).to(DEVICE)

# Load UNet LoRA
pipe_lora.unet = PeftModel.from_pretrained(pipe_lora.unet, UNET_LORA_WEIGHTS).to(DEVICE)

# Optionally load Text Encoder LoRA if available
if os.path.exists(TEXT_ENCODER_LORA_WEIGHTS):
    pipe_lora.text_encoder = PeftModel.from_pretrained(pipe_lora.text_encoder, TEXT_ENCODER_LORA_WEIGHTS).to(DEVICE)
    print("Loaded LoRA weights for Text Encoder.")

pipe_lora.scheduler = DPMSolverMultistepScheduler.from_config(pipe_lora.scheduler.config)
pipe_lora.enable_xformers_memory_efficient_attention()
pipe_lora.enable_attention_slicing()

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


Loaded LoRA weights for Text Encoder.


# Generate and save images

In [ ]:
for i, prompt in enumerate(tqdm(PROMPTS, desc="Generating images")):
    # Base SD
    image_base = pipe_base(
        prompt,
        height=RESOLUTION,
        width=RESOLUTION,
        num_inference_steps=NUM_INFERENCE_STEPS,
        guidance_scale=GUIDANCE_SCALE
    ).images[0]
    image_base.save(Path(OUTPUT_DIR) / f"prompt_{i}_base.png")

    # LoRA SD
    image_lora = pipe_lora(
        prompt,
        height=RESOLUTION,
        width=RESOLUTION,
        num_inference_steps=NUM_INFERENCE_STEPS,
        guidance_scale=GUIDANCE_SCALE
    ).images[0]
    image_lora.save(Path(OUTPUT_DIR) / f"prompt_{i}_lora.png")

print(f"Images saved to {OUTPUT_DIR}")

Generating images:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Generating images:  50%|█████     | 1/2 [00:19<00:19, 19.25s/it]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Generating images: 100%|██████████| 2/2 [00:31<00:00, 15.85s/it]

Images saved to /content/drive/MyDrive/Colab Notebooks/sd_lora/testing/smoke_test/unet_te/generated_test_images
